In [ ]:
import pickle


with open("/content/drive/MyDrive/Rag Chat/embeddings.pkl", "wb") as f:
    pickle.dump(all_embeddings, f)


In [ ]:
from langchain.vectorstores import FAISS

# Extract just the text from the documents
texts = [doc.page_content for doc in document]

# Create (text, embedding) pairs
text_embedding_pairs = list(zip(texts, all_embeddings))

# Create FAISS index from precomputed embeddings
db = FAISS.from_embeddings(text_embedding_pairs, model)
DB_FAISS_PATH='/content/drive/MyDrive/Rag Chat'
db.save_local(DB_FAISS_PATH)


In [ ]:
#Load model

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""
model_name = "HuggingFaceH4/zephyr-7b-beta"
def load_model(model_name):
    llm = HuggingFaceEndpoint(
        repo_id=model_name,
        temperature=0.5,
        max_new_tokens=512,
    )
    return llm


llm = load_model(model_name)


In [ ]:
def custom_prompt():
    prompt_template = """Use the following context to answer the question at the end. 
Answer only with "yes" or "no" — nothing else. 
If you don't know the answer, say "no". 
{context}
Question: {question}
Answer:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    return PROMPT


In [ ]:
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

DB_FAISS_PATH = '/content/drive/MyDrive/Rag Chat'
db_ = FAISS.load_local(DB_FAISS_PATH, embeddings_model, allow_dangerous_deserialization=True)


In [ ]:
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

DB_FAISS_PATH = '/content/drive/MyDrive/Rag Chat'
db_ = FAISS.load_local(DB_FAISS_PATH, embeddings_model, allow_dangerous_deserialization=True)


In [ ]:
#Create QA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db_.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True,
    chain_type_kwargs={'prompt': custom_prompt()}
)


In [ ]:
# user_query = input('Write Query: ')
# response = qa_chain.invoke({"query": user_query})
print("Answer:")
listt = [
    {
        "ID": "1",
        "Statement": "Rama is the eldest son of King Dasharatha.",
        "Truth": "TRUE",
        "Reference": "Bala Kanda, Sarga 18, Shloka 24"
    },
    {
        "ID": "2",
        "Statement": "Sita was discovered by King Janaka in a furrow during ploughing and was later adopted by him.",
        "Truth": "TRUE",
        "Reference": "Bala Kanda, Sarga 66, Shloka 14"
    },
    {
        "ID": "3",
        "Statement": "Lakshmana, Ramaâs devoted younger brother, accompanied him into exile.",
        "Truth": "TRUE",
        "Reference": "Ayodhya Kanda, Sarga 31, Shloka 30"
    },
    {
        "ID": "4",
        "Statement": "Bharata, another brother of Rama, revered him and ruled as regent in his absence.",
        "Truth": "TRUE",
        "Reference": "Ayodhya Kanda, Sarga 115, Shloka 22"
    },
    {
        "ID": "5",
        "Statement": "Hanuman was the best minister of Sugreeva.",
        "Truth": "TRUE",
        "Reference": "Kishkindha Kanda, Sarga 6, Shloka 2"
    },
    {
        "ID": "6",
        "Statement": "Ravana, the demon king of Lanka, abducted Sita, setting in motion the events of the epic.",
        "Truth": "TRUE",
        "Reference": "Aranya Kanda, Sarga 49, Shloka 20"
    },
    {
        "ID": "7",
        "Statement": "On seeing Hanuman, Sita lost her consciousness for a long time.",
        "Truth": "TRUE",
        "Reference": "Sundara Kanda, Sarga 32, Shloka 8"
    },
    {
        "ID": "8",
        "Statement": "Nila, a general of the monkey army, was rendered unconscious by a mystic arrow shot by Ravana.",
        "Truth": "TRUE",
        "Reference": "Yuddha Kanda, Sarga 59, Shloka 89"
    },
    {
        "ID": "9",
        "Statement": "Nala constructed a bridge for Rama to cross over to Lanka.",
        "Truth": "TRUE",
        "Reference": "Yuddha Kanda, Sarga 27, Shloka 61"
    },
    {
        "ID": "10",
        "Statement": "The sage Vishwamitra mentored Rama during his early life, imparting knowledge of warfare and ritual.",
        "Truth": "TRUE",
        "Reference": "Bala Kanda, Sarga 22"
    },
    {
        "ID": "11",
        "Statement": "Sita herself built the bridge to Lanka with the help of the monkey army.",
        "Truth": "FALSE",
        "Reference": ""
    },
    {
        "ID": "12",
        "Statement": "Lakshmana was a disciple of Ravana and learned warfare techniques from him.",
        "Truth": "FALSE",
        "Reference": ""
    },
    {
        "ID": "13",
        "Statement": "Bharata was originally from Lanka and was later adopted by King Dasharatha.",
        "Truth": "FALSE",
        "Reference": ""
    },
    {
        "ID": "14",
        "Statement": "After meeting Sita, Hanuman lost his ability to speak entirely.",
        "Truth": "FALSE",
        "Reference": ""
    },
    {
        "ID": "15",
        "Statement": "Ravana is depicted solely as a peaceful poet who never engaged in any battles.",
        "Truth": "FALSE",
        "Reference": ""
    },
    {
        "ID": "16",
        "Statement": "Ramaâs exile lasted only one year rather than the traditionally recounted fourteen years.",
        "Truth": "FALSE",
        "Reference": ""
    },
    {
        "ID": "17",
        "Statement": "Sita and Rama were married in Lanka with Ravana acting as the priest for their wedding.",
        "Truth": "FALSE",
        "Reference": ""
    },
    {
        "ID": "18",
        "Statement": "Lakshmana is not mentioned at all in the Valmiki Ramayana.",
        "Truth": "FALSE",
        "Reference": ""
    },
    {
        "ID": "19",
        "Statement": "Bharata built a grand palace in Lanka following Ramaâs victory over Ravana.",
        "Truth": "FALSE",
        "Reference": ""
    },
    {
        "ID": "20",
        "Statement": "Hanuman was the son of Ravana.",
        "Truth": "FALSE",
        "Reference": ""
    }
]
for item in listt:
  print(item['Statement'])
  response = qa_chain.invoke({"query": item['Statement']})
  print("Answer:")
  print(response['result'])




In [ ]:
try:
    user_query = input('Write Query: ')
    response = qa_chain.invoke({"query": user_query})
    print("Answer:")
    print(response['result'])
    
    # Optionally print source documents
    if 'source_documents' in response:
        print("Source documents:")
        for i, doc in enumerate(response['source_documents']):
            print(f"Document {i+1}: {doc.page_content[:200]}...")
            
except Exception as e:
    print(f"Error during query processing: {str(e)}")
    print("Please check your FAISS index path and model availability.")





# # Create RetrievalQA chain


# # Query loop with error handling
try:
    user_query = input('Write Query: ')
    response = qa_chain.invoke({"query": user_query})
    print("\nAnswer:")
    print(response['result'])
    
    # Optionally print source documents
    if 'source_documents' in response:
        print("\nSource documents:")
        for i, doc in enumerate(response['source_documents']):
            print(f"Document {i+1}: {doc.page_content[:200]}...")
            
except Exception as e:
    print(f"Error during query processing: {str(e)}")
    print("Please check your FAISS index path and model availability.")